In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load historical stock repurchase dataset
df_stock_historical = pd.read_excel("5 parameters 2019-2023.xlsx")

# Define input features (excluding 2023, as it's the target for training)
X_columns = ["2019", "2020", "2021", "2022"]
y_column = "2023"

# Extract input features (X) and target variable (y)
X = df_stock_historical[X_columns].astype(float).fillna(0).values
y = df_stock_historical[y_column].astype(float).fillna(0).values

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define Neural Network model
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1)  # Output layer for predicting repurchase value
])

# Compile model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train model
model.fit(X_train_scaled, y_train, epochs=100, batch_size=8, validation_data=(X_test_scaled, y_test), verbose=1)

# Prepare input data for forecasting (latest known values from 2023)
X_future = X.copy()
X_future_scaled = scaler.transform(X_future)

# Initialize DataFrame for rolling predictions
df_predictions = df_stock_historical[["Company_name"]].copy()
df_predictions["Parameters"] = "repurchase of common stock"

# Perform rolling predictions for 2024-2033 using NN model
for year in range(2024, 2034):
    y_pred = model.predict(X_future_scaled).flatten()
    df_predictions[str(year)] = y_pred
    
    # Update input for the next year (rolling effect)
    X_future = np.hstack([X_future[:, 1:], y_pred.reshape(-1, 1)])

# Save the Neural Network-based predictions
df_predictions.to_excel("repurchase_predictions_nn_2024_2033.xlsx", index=False)

print("Predictions saved as repurchase_predictions_nn_2024_2033.xlsx")


KeyError: "None of [Index(['2019', '2020', '2021', '2022'], dtype='object')] are in the [columns]"